In [ ]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [ ]:
import pandas as pd
import unidecode

In [ ]:
url = 'https://www.sunedu.gob.pe/lista-universidades/'

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
driver.get(url)
print ('url: ', driver.current_url)
print('title: ', driver.title)
driver.maximize_window()

In [ ]:
publicas_path = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div[1]/div/div/div/div[1]/ul/li[1]/a')
publicas_path.click()

In [ ]:
table1_path = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div[1]/div/div/div/div[1]/div/div[1]/div')
table1_html = table1_path.get_attribute('innerHTML')

In [ ]:
#reading table using pandas
table1 = pd.read_html(table1_html)
table_publicas = table1[0].copy()

In [ ]:
# insertar nuevas columnas
table_publicas['TIPO'] = "Pública"
table_publicas['CONDICIÓN JURÍDICA'] = "NaN"
table_publicas.rename(columns = {'DISPOSITIVO LEGAL DE LICENCIAMIENTO': 'DISPOSITIVO LEGAL DE LICENCIAMIENTO O DENEGATORIA'} , inplace=True)
table_publicas

In [ ]:
privadas_path = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div[1]/div/div/div/div[1]/ul/li[2]/a')
privadas_path.click()

In [ ]:
table2_path = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div')
table2_html = table2_path.get_attribute('innerHTML')

In [ ]:
table2 = pd.read_html(table2_html)
table_privadas = table2[0].copy()

In [ ]:
table_privadas['TIPO'] = "Privada"
table_privadas

In [ ]:
print(table_publicas.shape)
print(table_privadas.shape)

In [ ]:
#pd.concat
table_universidades = pd.concat([table_publicas, table_privadas])
table_universidades